In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd

nltk.download('stopwords')
stop = stopwords.words('english')
stemmer = nltk.stem.PorterStemmer()

df_text = pd.read_csv('rnews_apple.txt', sep=': \"', header=None, names=['date', 'text'], dtype='str', engine='python')
df_text['text_without_stopwords'] = df_text['text'].apply(lambda x: ' '.join([stemmer.stem(word).lower() for word in x.split() if word not in (stop)]))
print(df_text[:4])


[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


       date                                               text  \
0  01012013              Armed robbers hit Paris Apple store",   
1  01022013  UPDATE 1-Judge rejects part of Apple App Store...   
2  01042013  US STOCKS-Data helps pace Wall St higher, but ...   
3  01062013  Riches in niches: U.S. cops, in-flight movies ...   

                              text_without_stopwords  
0                   arm robber hit pari appl store",  
1  updat 1-judg reject part appl app store suit v...  
2  us stocks-data help pace wall st higher, appl ...  
3  rich niches: u.s. cops, in-flight movi may mod...  


In [2]:
import numpy as np

df = pd.read_csv('all_stocks_5yr.csv')
df = df.loc[(df['Name'] == "AAPL")]
df = df.drop('volume', 1)
df = df.drop('high', 1)
df = df.drop('low', 1)
df = df.drop('Name', 1)
print('df.shape',df.shape)
print('df.shape',df[:3])

df['next_day_open'] = df['open'].shift(-1)
df['next_day_close'] = df['close'].shift(-1)
df['label'] = np.where((df['next_day_close'] > df['close']), 1, 0)
df = df.sort_values('date').reset_index(drop=True)
df = df.drop('next_day_close', 1)
df = df.dropna()
print('df.shape',df.shape)
print('df.shape',df[:3])

df.shape (1259, 3)
df.shape             date     open    close
1259  2013-02-08  67.7142  67.8542
1260  2013-02-11  68.0714  68.5614
1261  2013-02-12  68.5014  66.8428
df.shape (1258, 5)
df.shape          date     open    close  next_day_open  label
0  2013-02-08  67.7142  67.8542        68.0714      1
1  2013-02-11  68.0714  68.5614        68.5014      0
2  2013-02-12  68.5014  66.8428        66.7442      0


In [3]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
import datetime
from tqdm import tqdm
import copy

class MainRNN():
    def __init__(self):
        self.data_time_range=1
        self.seq_size=self.data_time_range
        self.hidden_layer=1
        self.output_feature_size=1
        self.epochs=500
        self.batch_size=2

    def get_formated_data(self):
                
        count_vectorizer = CountVectorizer(decode_error='ignore',binary=False)
        X_text = count_vectorizer.fit_transform(df_text['text_without_stopwords'])
        X_text = X_text.toarray()
        
        print(df[:3])

        X = []
        Y = []
        for index, row in tqdm(df.iterrows()):
            row_data_array = row.values[1:-1]
            row_day = row['date'].split('-')[2]
            row_mon = row['date'].split('-')[1]
            row_year = row['date'].split('-')[0]
            text_index = df_text[(df_text['date'] == row_mon + row_day + row_year)].index.values.astype(int)
            if len(text_index) > 0:
                text_index = text_index[0]

                if (text_index) >= 0: 
                    row_data_array = np.append(row_data_array, np.array(X_text[text_index], dtype=np.int))
                else:
                    _, shape_for_use = np.array(X_text).shape
                    row_data_array = np.append(row_data_array, np.zeros(shape_for_use, dtype=np.int))
            else: 
                _, shape_for_use = np.array(X_text).shape
                row_data_array = np.append(row_data_array, np.zeros(shape_for_use, dtype=np.int))
                
            X.append([row_data_array])
            Y.append([row['label']])
        
        print(X[1], Y[1])
        print('before X.shape', len(X))
        print('before Y.shape', len(Y))
        
        return np.array(X), np.array(Y)


    def process_train(self, X, Y):

        # get shape X (N, T, D)
        X_sample_size, X_seq_size, X_features_size = X.shape
        print('X_features_size', X_features_size)

        # get shape Y (K)
        Y_sample_size = Y.shape

        # init weight and bias
        weights = tf.Variable(tf.random_normal([self.hidden_layer, self.output_feature_size]))
        biases = tf.Variable(tf.random_normal([self.output_feature_size]))

        # placeholder for graph input
        tfX = tf.placeholder(tf.float32, shape=[None, X_seq_size, X_features_size], name='inputX')
        tfY = tf.placeholder(tf.float32, shape=[None, self.output_feature_size], name='inputY')

        # transposeX
        tfX = tf.transpose(tfX, [1, 0, 2])

        # define lstm cell
        lstmCell = tf.nn.rnn_cell.BasicLSTMCell(self.hidden_layer)

        # create RNN unit
        outputs, states = tf.nn.dynamic_rnn(cell=lstmCell, inputs=tfX, dtype=tf.float32)

        # get rnn output
        outputs = tf.stack(outputs)

        # transpose output back
        outputs = tf.transpose(outputs, [1, 0, 2])
        # outputs = tf.reshape(outputs, [outputs.get_shape()[-1], self.hidden_layer])

        # Hack to build the indexing and retrieve the right output.
        # Start indices for each sample
        index = tf.range(0, tf.shape(outputs)[0]) * X_seq_size + (X_seq_size - 1)
        # Indexing
        outputs = tf.gather(tf.reshape(outputs, [-1, self.hidden_layer]), index)

        # prediction
        prediction = tf.matmul(outputs, weights) + biases
        prediction = tf.nn.softmax(prediction)

        # cost function
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=prediction, labels=tfY))
        
        
        
        # optimizer
        optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(loss)

        # cost[] and accuracies[]
        epochs_correct = []
        epochs_tp = []
        epochs_fp = []
        epochsl=[]
        epochs_cost_sum=[]

        # global init
        init = tf.global_variables_initializer()

        first=True

        # start training
        with tf.Session() as sess:
            saver = tf.train.Saver()
            sess.run(init)

            for epoch in range(self.epochs):
                print('===== EPOCH ======: ', epoch)
                if not first:
                    saver.restore(sess,"./trained_model")
                    first=False
                
                correctc=0
                tpc=0
                fpc=0
                cost_sum=0
                X, Y = shuffle(X, Y)
                for batch in tqdm(range(X_sample_size - self.batch_size)):
                    batchX = X[batch:batch+self.batch_size]
                    batchY = Y[batch:batch+self.batch_size]

                    _, cost_out, prediction_out = sess.run([optimizer, loss, prediction], feed_dict={tfX: batchX.reshape(X_seq_size, self.batch_size, X_features_size), tfY: batchY.reshape(self.batch_size, self.output_feature_size)})
                    # evaluation
                    cost_sum+=cost_out
                    
                    correct_pred = 1 if prediction_out[1] == Y[batch+1] else 0
                    tp_out = 1 if prediction_out[1] == 1 and Y[batch+1] == 1 else 0
                    fp_out = 1 if prediction_out[1] == 1 and Y[batch+1] == 0 else 0
                    
                    correctc+=correct_pred
                    tpc+=tp_out
                    fpc+=fp_out
                epochs_cost_sum.append(cost_sum)
                saver.save(sess, "./trained_model")
                print('accuracy:', correctc/X_sample_size)
                print('cost_sum:', cost_sum)
                epochsl.append(epoch)
                epochs_correct.append(correctc)
                epochs_tp.append(tpc)
                epochs_fp.append(fpc)
                

            np.random.seed(0)
#             plt.plot(epochsl, epochs_fp, 'r--', epochsl, epochs_tp, 'bs', epochsl, epochs_correct, 'g^')
            plt.plot(epochsl,epochs_cost_sum, 'g^')
            plt.savefig('costs.png')


    def run_prediction(self):
        print('start time:', datetime.datetime.now())
        X, Y = self.get_formated_data()
        self.process_train(X, Y)
        print('end time:', datetime.datetime.now())


if __name__ == '__main__':
    MainRNN().run_prediction()


/Users/apple/anaconda/envs/HappyNewYear/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/apple/anaconda/envs/HappyNewYear/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
0it [00:00, ?it/s]

start time: 2019-05-09 22:19:27.092126
         date     open    close  next_day_open  label
0  2013-02-08  67.7142  67.8542        68.0714      1
1  2013-02-11  68.0714  68.5614        68.5014      0
2  2013-02-12  68.5014  66.8428        66.7442      0


1258it [00:11, 112.54it/s]


[array([68.0714, 68.5614, 68.5014, ..., 0, 0, 0], dtype=object)] [0]
before X.shape 1258
before Y.shape 1258
X_features_size 2476


/Users/apple/anaconda/envs/HappyNewYear/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
  0%|          | 0/1256 [00:00<?, ?it/s]

...

accuracy: 0.5151033386327504
cost_sum: 1001.456735253334
===== EPOCH ======:  498


  0%|          | 0/1256 [00:00<?, ?it/s]

accuracy: 0.5158982511923689
cost_sum: 999.9567341208458
===== EPOCH ======:  499


100%|██████████| 1256/1256 [00:03<00:00, 335.68it/s]


accuracy: 0.5151033386327504
cost_sum: 1001.4567351341248
end time: 2019-05-09 23:23:42.018323
